# Preparation and Cleaning of the FIES & LFS 2021 Dataset

This notebook documents the workflow for preparing and cleaning the Family Income and Expenditure Survey (FIES) and Labor Force Survey (LFS) 2021 dataset, focusing on Region 13. The steps include data loading, merging, filtering, cleaning, variable transformation, indicator construction, and saving the final dataset for analysis.

---

## Workflow Overview

1. **Data Loading**
    - Import libraries (`pandas`, `numpy`, `importlib`, `os`).
    - Load individual-level and household-level CSVs.
    - Load external poverty indicator data.

2. **Data Merging**
    - Merge individual and household DataFrames on shared keys (`W_REGN`, `W_PROV`, `SEQUENCE_NO`).
    - Filter for Region 13 (`W_REGN == 13`).
    - Merge poverty indicators.

3. **Variable Cleaning and Transformation**
    - Clean proxy variables: convert types, handle missing values, remove leading zeros, and standardize formats.
    - Recode education levels using a custom function.
    - Fill missing values contextually based on age and other conditions.
    - Map categorical codes to descriptive labels.

4. **Indicator Construction**
    - Construct household-level and individual-level indicators (education buckets, age buckets, occupation codes, worker counts).
    - Create indicators for domestic helpers and household head characteristics.

5. **Final DataFrame Refinement**
    - Remove individual-level columns and unnecessary variables.
    - Merge household-level indicators.
    - Convert selected columns to categorical types.
    - Remove duplicates.

6. **Saving the Cleaned Dataset**
    - Save the final DataFrame (`df_21`) as a CSV file for further analysis.

---

## Outputs

- **Cleaned DataFrame (`df_21`)**: Merged, filtered, and labeled data for Region 13, with household and individual indicators.
- **CSV File**: Saved for downstream analysis.

---

## Purpose

This notebook ensures the FIES & LFS 2021 data is:
- Consistently formatted and labeled
- Ready for socioeconomic analysis
- Documented for reproducibility and transparency


In [88]:
import pandas as pd
import numpy as np
import importlib
import os

In [89]:
# Create output folder if it doesn't exist

output_folder = "output"
os.makedirs(output_folder, exist_ok=True)

## Loading and Merging the Dataset

This section covers the initial preparation of the FIES and LFS 2021 data:

**Inputs:**
- Individual-level CSV (household members)
- Household-level CSV (summary)
- External CSV (poverty indicators)

**Steps:**
1. Load CSVs into DataFrames.
2. Merge on shared keys (`W_REGN`, `W_PROV`, `SEQUENCE_NO`).
3. Filter for Region 13 (`W_REGN == 13`).
4. Merge poverty indicators.

**Output:**  
A cleaned, merged DataFrame (`df_21`) for Region 13, ready for analysis.

### FIES LFS 2021 HHLD_SUMMARY

In [ ]:
# The dataset is full of errors and can't be used as it is. We will parse first as text and filter
# the rows that contain the start with 13

file_path = './data/FIES&LFS/FIES LFS Merge 2021/FIES-LFS PUF 2021 - HHLD_SUMMARY.CSV'

with open(file_path, 'r') as f:
    lines = f.readlines()

print(len(lines))
# Filter the rows that start with 13
titles =  lines[0]
lines = [line for line in lines if line.startswith('13')]
print(len(lines))

# Save the filtered rows to a new file
filtered_file_path = './data/FIES&LFS/FIES LFS Merge 2021/FIES-LFS PUF 2021 - HHLD_SUMMARY_filtered.CSV'

with open(filtered_file_path, 'w') as f:
    f.write(titles)
    f.writelines(lines)


165030
21689


In [91]:
# Load the CSV file into a DataFrame
df_FIES21LSF = pd.read_csv(filtered_file_path, on_bad_lines='warn')

# Display the first few rows of the DataFrame
df_FIES21LSF.head()

/var/folders/x5/pd9ssg3n4cd2x2yf5kvq955r0000gp/T/ipykernel_31675/3039528782.py:2: ParserWarning: Skipping line 15258: expected 103 fields, saw 105
Skipping line 15259: expected 103 fields, saw 104
Skipping line 15293: expected 103 fields, saw 105

  df_FIES21LSF = pd.read_csv(filtered_file_path, on_bad_lines='warn')


,W_REGN,W_PROV,SEQUENCE_NO,FSIZE,REG_SAL,SEASON_SAL,WAGES,NETSHARE,CASH_ABROAD,CASH_DOMESTIC,...,S150117_REF_QTY,S150118_WASH_QTY,S150119_AIRCON_QTY,S150120_CAR_QTY,S150121_LANDLINE_QTY,S150122_CELL_QTY,S150123_COMP_QTY,S150124_OVEN_QTY,S150125_BANCA_QTY,S150126_MOTOR_QTY
0,13,39,108973,6.5,337000,0,337000,0,0,2000,...,01,01,01,,01,01,01,01,,
1,13,39,108974,1.0,0,0,0,0,30000,3500,...,,,,,,01,,01,,
2,13,39,108975,4.0,0,0,0,0,0,42000,...,01,01,,,,03,,00,,
3,13,39,108976,5.0,259908,77328,337236,0,0,11500,...,01,01,00,00,00,04,00,00,00,00
4,13,39,108977,5.0,0,398656,398656,0,0,16800,...,00,01,,,,05,,00,,


### FIES LFS 2021 HHMEM

In [ ]:
# The dataset is full of errors and can't be used as it is. We will parse first as text and filter
# the rows that contain the start with 13

file_path = './data/FIES&LFS/FIES LFS Merge 2021/FIES-LFS PUF 2021 - HHMEM.CSV'

with open(file_path, 'r', encoding='latin1') as f:
    lines = f.readlines()

print(len(lines))
# Filter the rows that start with 13
titles =  lines[0]
lines = [line for line in lines if line.startswith('13')]
print(len(lines))

# Save the filtered rows to a new file
filtered_file_path = './data/FIES&LFS/FIES LFS Merge 2021/FIES-LFS PUF 2021 - HHMEM_filtered.CSV'

with open(filtered_file_path, 'w') as f:
    f.write(titles)
    f.writelines(lines)


716394
90260


In [93]:
# Load the CSV file into a DataFrame
df_FIES21LSF2 = pd.read_csv(filtered_file_path, on_bad_lines='warn')

# Display the first few rows of the DataFrame
df_FIES21LSF2.head()

/var/folders/x5/pd9ssg3n4cd2x2yf5kvq955r0000gp/T/ipykernel_31675/3675036660.py:2: ParserWarning: Skipping line 1763: expected 53 fields, saw 54
Skipping line 2938: expected 53 fields, saw 54
Skipping line 3003: expected 53 fields, saw 54
Skipping line 6804: expected 53 fields, saw 54
Skipping line 10199: expected 53 fields, saw 54
Skipping line 10290: expected 53 fields, saw 54
Skipping line 11457: expected 53 fields, saw 54
Skipping line 11686: expected 53 fields, saw 54
Skipping line 12340: expected 53 fields, saw 54
Skipping line 13756: expected 53 fields, saw 54
Skipping line 14242: expected 53 fields, saw 54
Skipping line 14243: expected 53 fields, saw 54
Skipping line 14390: expected 53 fields, saw 54
Skipping line 14891: expected 53 fields, saw 54
Skipping line 15169: expected 53 fields, saw 54
Skipping line 15296: expected 53 fields, saw 56

  df_FIES21LSF2 = pd.read_csv(filtered_file_path, on_bad_lines='warn')
/var/folders/x5/pd9ssg3n4cd2x2yf5kvq955r0000gp/T/ipykernel_31675/36

,W_REGN,W_PROV,SEQUENCE_NO,LC101_LNO,LC03_REL,LC04_SEX,LC05_AGE,LC05A_5OVER,LC06_MSTAT,LC07_HGC_LEVEL,...,LC36_AVAIL,LC37_WILLING,LC38_PREVJOB,LC39_YEAR,LC39_MONTH,LC41_POCC,LC42INDUSTRY,LC43_QKB,NEWEMPSTAT,PWGTPRV
0,13,39,108973,1,1,1,59,,2,2,...,,,,,,,...,,1,185.0937
1,13,39,108973,2,2,2,56,,2,2,...,,,,,,,...,,1,167.8637
2,13,39,108973,3,3,2,18,,1,6,...,,,2,,,,...,,3,200.4619
3,13,39,108973,4,3,2,36,,1,6,...,,,,,,,...,,1,192.4503
4,13,39,108973,5,6,2,2,,,,...,,,,,,,...,,,208.6799


### Merge

In [94]:
# Merge the two dataframes on common columns
merged_df = pd.merge(df_FIES21LSF, df_FIES21LSF2, on=['W_REGN', 'W_PROV', 'SEQUENCE_NO'])

# Drop rows where W_REGN is not equal to 13
df_21 = merged_df[merged_df['W_REGN'] == 13]

df_21

,W_REGN,W_PROV,SEQUENCE_NO,FSIZE,REG_SAL,SEASON_SAL,WAGES,NETSHARE,CASH_ABROAD,CASH_DOMESTIC,...,LC36_AVAIL,LC37_WILLING,LC38_PREVJOB,LC39_YEAR,LC39_MONTH,LC41_POCC,LC42INDUSTRY,LC43_QKB,NEWEMPSTAT,PWGTPRV
0,13,39,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,,1,185.0937
1,13,39,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,,1,167.8637
2,13,39,108973,6.5,337000,0,337000,0,0,2000,...,,,2,,,,...,,3,200.4619
3,13,39,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,,1,192.4503
4,13,39,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,,,208.6799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90203,13,76,130660,4.5,194394,15600,209994,0,0,12000,...,,,,,,,...,,,194.3881
90204,13,76,130660,4.5,194394,15600,209994,0,0,12000,...,,,,,,,...,,,181.7025
90205,13,76,130660,4.5,194394,15600,209994,0,0,12000,...,,,,,,,...,,,194.3881
90206,13,76,130661,2.0,0,0,0,0,90000,134000,...,,,2,,,,...,,3,149.2373


In [95]:
with open(os.path.join(output_folder, 'columns_list_21.txt'), 'w') as f:
    for column in df_21.columns:
        f.write(f"{column}\n")

## Remapping of the Highest Completed Grades

`LC07_GRADE` is recoded using the `recode_edu` function from the `education_recode` module to standardize educational attainment categories in `df_21`.

**Purpose:**  
- Groups education levels for easier analysis.
- Ensures consistent, interpretable data.

In [96]:

edu = importlib.import_module("education_recode")   # gives you edu.MAP  &  edu.recode_edu

mod = {}
exec(open("education_recode.py").read(), mod)   # gives mod["recode_edu"]

df_21["LC07_GRADE"] = df_21["LC07_GRADE"].apply(mod["recode_edu"])


## Cleaning and Labeling Proxy Variables

This section outlines the cleaning and labeling of proxy variables in `df_21`, representing key household and individual characteristics for socioeconomic analysis.

**Steps:**
- **Identify Proxies:** Select columns as proxies (e.g., education, occupation, income, poverty line).
- **Clean Data:** Convert types, handle missing values, remove leading zeros, and standardize formats.
- **Label Variables:** Map codes to descriptive labels and fill missing values contextually.
- **Verify:** Print summary statistics and NaN counts.

**Output:**  
A cleaned and labeled `df_21` DataFrame, ready for analysis.

In [97]:
df_21['LC14_PROCC'].value_counts() 

# Keep only the first digit (as a string) of each 4-digit code, including leading zero
df_21['LC14_PROCC'] = df_21['LC14_PROCC'].astype(str).str.zfill(4).str[0]
df_21['LC14_PROCC'].value_counts()

LC14_PROCC
     54182
5     9824
9     7137
4     5574
8     3420
7     2950
2     2537
3     2455
1     2018
6       58
0       53
Name: count, dtype: int64

In [98]:
# Define the mapping of old variable names to new variable names
rename_dict = {
    'S150111_WATER_SUPPLY': 'H150111_WATER_SUPPLY',
    'S150110_ELECTRICITY': 'H150110_ELECTRICITY',
    'S150104_TENURE_STA':  'H150104_TENURE_STA',
    'S150103A_MAIN':   'H150103A_MAIN',
    'S150103_WALLS': 'H150103_WALLS',
    'S150102_ROOF': 'H150102_ROOF',
    'S150101_BLDG_TYPE': 'H150101_BLDG_TYPE',
    'S150113_RADIO_QTY': 'H150113_RADIO_QTY',
    'S150114_TV_QTY': 'H150114_TV_QTY',
    'S150115_VCD_QTY': 'H150115_VCD_QTY',
    'S150116_STEREO_QTY': 'H150116_STEREO_QTY',
    'S150117_REF_QTY': 'H150117_REF_QTY',
    'S150118_WASH_QTY': 'H150118_WASH_QTY',
    'S150119_AIRCON_QTY': 'H150119_AIRCON_QTY',
    'S150120_CAR_QTY': 'H150120_CAR_QTY',
    'S150121_LANDLINE_QTY': 'H150121_LANDLINE_QTY',
    'S150122_CELL_QTY': 'H150122_CELL_QTY',
    'S150123_COMP_QTY': 'H150123_COMP_QTY',
    'S150124_OVEN_QTY': 'H150124_OVEN_QTY',
    'S150125_BANCA_QTY': 'H150125_BANCA_QTY',
    'S150126_MOTOR_QTY': 'H150126_MOTOR_QTY',
    'S150109_TOILET': 'H150109_TOILET',
    'S150108_HSE_ALTERATION' : 'H150108_HSE_ALTERATION'
}


# Rename the columns in the DataFrame
df_21.rename(columns=rename_dict, inplace=True)

# Display the first few rows of the updated DataFrame to verify the changes
df_21.head()

,W_REGN,W_PROV,SEQUENCE_NO,FSIZE,REG_SAL,SEASON_SAL,WAGES,NETSHARE,CASH_ABROAD,CASH_DOMESTIC,...,LC36_AVAIL,LC37_WILLING,LC38_PREVJOB,LC39_YEAR,LC39_MONTH,LC41_POCC,LC42INDUSTRY,LC43_QKB,NEWEMPSTAT,PWGTPRV
0,13,39,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,,1,185.0937
1,13,39,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,,1,167.8637
2,13,39,108973,6.5,337000,0,337000,0,0,2000,...,,,2,,,,...,,3,200.4619
3,13,39,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,,1,192.4503
4,13,39,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,,,208.6799


In [99]:
check_list = [
    'W_REGN',
    'W_PROV',
    'SEQUENCE_NO',
    'FSIZE',
    'PCINC',
    'URB',
    'RPROV',
    'RPSU',
    'BWEIGHT',
    'RFACT',
    'RFACT_POP',
    'H150101_BLDG_TYPE',
    'H150102_ROOF',
    'H150103_WALLS',
    'H150103A_MAIN',
    'H150104_TENURE_STA',
    'H150109_TOILET',
    'H150110_ELECTRICITY',
    'H150111_WATER_SUPPLY',
    'H150113_RADIO_QTY',
    'H150114_TV_QTY',
    'H150115_VCD_QTY',
    'H150116_STEREO_QTY',
    'H150117_REF_QTY',
    'H150118_WASH_QTY',
    'H150119_AIRCON_QTY',
    'H150120_CAR_QTY',
    'H150121_LANDLINE_QTY',
    'H150122_CELL_QTY',
    'H150123_COMP_QTY',
    'H150124_OVEN_QTY',
    'H150125_BANCA_QTY',
    'H150126_MOTOR_QTY',
    'LC101_LNO',
    'LC03_REL',
    'LC04_SEX',
    'LC05_AGE',
    'LC06_MSTAT',
    'LC07_GRADE',
    'LC08_CURSCH',
    'LC10_CONWR',
    'LC14_PROCC',
    'LC16_PKB',
    'LC17_NATEM',
    'LC23_PCLASS',
    'H150108_HSE_ALTERATION',
    'LC24_PBASIS',
    'LC43_QKB',
    'LC12_JOB'
]

In [100]:
columns_to_clean = []

for variable in check_list:
    if df_21[variable].dtypes == 'int64':
        continue
    if df_21[variable].dtypes == 'float64':
        continue
    else:
        columns_to_clean.append(variable)
        continue

# Loop through the columns and clean the data
nan_counts = {}
obs_counts = {}

for column in columns_to_clean:

    obs_counts[column] = df_21[column].unique()

    # Convert to string to ensure consistency
    df_21[column] = df_21[column].astype(str)

    # Remove leading zeros, but keep standalone "0"
    df_21[column] = df_21[column].replace('00', '0')
    df_21[column] = df_21[column].replace('00000', '0')
    df_21[column] = df_21[column].apply(lambda x: '0' if x == '0' else x.lstrip('0'))

    df_21[column].replace("", np.NAN, inplace=True)
    df_21[column].replace(" ", np.NAN, inplace=True)
    df_21[column].replace("  ", np.NAN, inplace=True)
    df_21[column].replace("    ", np.NAN, inplace=True)
    df_21[column].replace("     ", np.NAN, inplace=True)
    
    # Count NaN values before dropping
    nan_counts[column] = df_21[column].isna().sum()
    
    df_21[column] = pd.to_numeric(df_21[column], errors='coerce')
    # df_FIES18.dropna(subset=[column], inplace=True)

    # Convert back to integer
    df_21[column] = df_21[column].astype(float)

    obs_counts[column + '_new'] = df_21[column].unique()

# Print the count of NaN values for each column
print("NaN values per column:")
for column, count in nan_counts.items():
    print(f"{column}: {count}")

/var/folders/x5/pd9ssg3n4cd2x2yf5kvq955r0000gp/T/ipykernel_31675/2741220862.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_21[column].replace("", np.NAN, inplace=True)
/var/folders/x5/pd9ssg3n4cd2x2yf5kvq955r0000gp/T/ipykernel_31675/2741220862.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alw

NaN values per column:
H150113_RADIO_QTY: 8350
H150114_TV_QTY: 684
H150115_VCD_QTY: 9746
H150116_STEREO_QTY: 8906
H150117_REF_QTY: 3596
H150118_WASH_QTY: 2294
H150119_AIRCON_QTY: 9069
H150120_CAR_QTY: 11395
H150121_LANDLINE_QTY: 10218
H150122_CELL_QTY: 361
H150123_COMP_QTY: 8271
H150124_OVEN_QTY: 7656
H150125_BANCA_QTY: 12905
H150126_MOTOR_QTY: 9663
LC06_MSTAT: 5461
LC08_CURSCH: 55137
LC10_CONWR: 21187
LC14_PROCC: 54182
LC16_PKB: 54182
LC17_NATEM: 54200
LC23_PCLASS: 54200
LC24_PBASIS: 61192
LC43_QKB: 77274
LC12_JOB: 42143


/var/folders/x5/pd9ssg3n4cd2x2yf5kvq955r0000gp/T/ipykernel_31675/2741220862.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_21[column].replace(" ", np.NAN, inplace=True)
/var/folders/x5/pd9ssg3n4cd2x2yf5kvq955r0000gp/T/ipykernel_31675/2741220862.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values al

In [101]:
# Label NaNs in LC08_CURSCH as 0 if the individual's age is < 5 or > 24
df_21.loc[(df_21['LC05_AGE'] < 5) | (df_21['LC05_AGE'] > 24), 'LC08_CURSCH'] = df_21['LC08_CURSCH'].fillna(0)
df_21.loc[(df_21['LC05_AGE'] <= 5), 'LC07_GRADE'] = df_21['LC07_GRADE'].fillna(1)
df_21.loc[(df_21['LC05_AGE'] <= 14) | (df_21['LC05_AGE'] > 65), 'LC17_NATEM'] = df_21['LC17_NATEM'].fillna(0)
df_21.loc[(df_21['LC05_AGE'] <= 14) | (df_21['LC05_AGE'] > 65), 'LC10_CONWR'] = df_21['LC10_CONWR'].fillna(0)
df_21.loc[(df_21['LC05_AGE'] <= 14) | (df_21['LC05_AGE'] > 65), 'LC23_PCLASS'] = df_21['LC23_PCLASS'].fillna(7)
df_21.loc[(df_21['LC05_AGE'] <= 14) | (df_21['LC05_AGE'] > 65), 'LC14_PROCC'] = df_21['LC14_PROCC'].fillna(0)
df_21.loc[(df_21['LC05_AGE'] <= 14) | (df_21['LC05_AGE'] > 65), 'LC12_JOB'] = df_21['LC12_JOB'].fillna(4)

# Display the first few rows of the updated dataframe to verify the changes
df_21

,W_REGN,W_PROV,SEQUENCE_NO,FSIZE,REG_SAL,SEASON_SAL,WAGES,NETSHARE,CASH_ABROAD,CASH_DOMESTIC,...,LC36_AVAIL,LC37_WILLING,LC38_PREVJOB,LC39_YEAR,LC39_MONTH,LC41_POCC,LC42INDUSTRY,LC43_QKB,NEWEMPSTAT,PWGTPRV
0,13,39,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,NaN,1,185.0937
1,13,39,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,NaN,1,167.8637
2,13,39,108973,6.5,337000,0,337000,0,0,2000,...,,,2,,,,...,NaN,3,200.4619
3,13,39,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,NaN,1,192.4503
4,13,39,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,NaN,,208.6799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90203,13,76,130660,4.5,194394,15600,209994,0,0,12000,...,,,,,,,...,NaN,,194.3881
90204,13,76,130660,4.5,194394,15600,209994,0,0,12000,...,,,,,,,...,NaN,,181.7025
90205,13,76,130660,4.5,194394,15600,209994,0,0,12000,...,,,,,,,...,NaN,,194.3881
90206,13,76,130661,2.0,0,0,0,0,90000,134000,...,,,2,,,,...,NaN,3,149.2373


In [102]:
df_21["H150110_ELECTRICITY"] = df_21["H150110_ELECTRICITY"].replace({2: 0})

In [103]:
list_vars = df_21.columns

## Individual and Household Indicator Construction

This section describes how household-level indicators are constructed by aggregating individual data from `df_21` using custom bucket functions and pandas group operations.

**Steps:**
- Copy `df_21` to `p` for processing.
- Define education and age buckets to ensure all categories are represented.
- Assign education and age buckets to individuals, one-hot encode, and aggregate counts by household (`SEQUENCE_NO`).
- Identify workers (`LC12_JOB == 1`), aggregate occupation codes, and count total workers per household.
- Merge all indicators into `merged_hh_data`, filling missing values with zeros.

**Output:**  
A household-level DataFrame (`merged_hh_data`) with counts for education, age, occupation, and workers, ready for analysis.


In [104]:
# --- 1. helper that buckets ONE code -----------------------------------------
def educ_band(code):
    if code in (0, 1, 10, 191, 192):
        return 'educ_none'                       # no formal schooling
    elif 310 <= code <= 500:
        return 'educ_primary'                   # grade 1 - grade 10
    elif 510 <= code <= 520 or 601 <= code <= 699:
        return 'educ_secondary'                 # grade 11-12 + post-sec certs
    elif 710 <= code <= 799 or 801 <= code <= 899:
        return 'educ_college'                   # associate / bachelor / college years
    elif 910 <= code <= 949:
        return 'educ_postgrad'                  # masters / doctorate
    else:
        return 'educ_other'                     # 999 or anything unexpected

# ---------- age buckets ----------
def age_band(a):
    if   a <= 5:   return 'age_0_5'
    elif a <=12:   return 'age_6_12'
    elif a <=17:   return 'age_13_17'
    elif a <=64:   return 'age_18_64'
    else:          return 'age_65p'

In [105]:
# Create a copy of the cleaned DataFrame for processing
p = df_21.copy()

# Define all possible buckets for education and age groups
all_educ_buckets = ['educ_none', 'educ_primary', 'educ_secondary', 'educ_college', 'educ_postgrad', 'educ_other']
all_age_buckets = ['age_0_5', 'age_6_12', 'age_13_17', 'age_18_64', 'age_65p']

# --- Education bucket counts per household ---
# For each household (SEQUENCE_NO), count the number of members in each education band
educ_cnt = (
  pd.get_dummies(p.set_index('SEQUENCE_NO')['LC07_GRADE'].apply(educ_band))
    .groupby('SEQUENCE_NO').sum()
    .reindex(columns=all_educ_buckets, fill_value=0)  # Ensure all buckets are present
)

# --- Age bucket counts per household ---
# For each household, count the number of members in each age band
age_cnt = (
  pd.get_dummies(p.set_index('SEQUENCE_NO')['LC05_AGE'].apply(age_band))
    .groupby('SEQUENCE_NO').sum()
    .reindex(columns=all_age_buckets, fill_value=0)  # Ensure all buckets are present
)

# --- Household roster ---
# List of all unique household IDs
all_hh = df_21['SEQUENCE_NO'].unique()

# --- Worker subset ---
# Select individuals with job code == 1 (workers)
workers = df_21[df_21['LC12_JOB'] == 1]

# --- Occupation counts per household ---
# For each household, count the number of workers in each occupation code (LC14_PROCC)
hh_occ_cnt = (
  pd.get_dummies(workers.set_index('SEQUENCE_NO')['LC14_PROCC'], prefix='occ4d')
    .groupby('SEQUENCE_NO').sum()
    .reindex(all_hh, fill_value=0)    # Include households with zero workers
)
hh_occ_cnt.rename(columns=lambda col: col.replace('.0', ''), inplace=True)

# --- Total worker count per household ---
hh_tot_workers = (
  workers.groupby('SEQUENCE_NO').size()
    .rename('n_workers')
    .reindex(all_hh, fill_value=0)    # Include households with zero workers
    .to_frame()
)

# --- Merge all household-level indicators ---
# Combine occupation counts, worker counts, age buckets, and education buckets into one DataFrame
merged_hh_data = (
  hh_occ_cnt
    .merge(hh_tot_workers, on='SEQUENCE_NO', how='outer')
    .merge(age_cnt, on='SEQUENCE_NO', how='outer')
    .merge(educ_cnt, on='SEQUENCE_NO', how='outer')
    .fillna(0)
)

# Display the first few rows of the merged household-level indicators
merged_hh_data.head()

,occ4d_1,occ4d_2,occ4d_3,occ4d_4,occ4d_5,occ4d_6,occ4d_7,occ4d_8,occ4d_9,n_workers,...,age_6_12,age_13_17,age_18_64,age_65p,educ_none,educ_primary,educ_secondary,educ_college,educ_postgrad,educ_other
SEQUENCE_NO,,,,,,,,,,,,,,,,,,,,,
108973,0,0,0,0,0,0,0,0,0,0,...,0,0,6,0,1,0,0,4,0,2
108974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
108975,0,0,0,0,0,0,0,0,0,0,...,0,1,2,1,0,0,1,0,0,3
108976,0,0,0,0,0,0,0,0,0,0,...,0,0,3,2,0,0,0,1,0,4
108977,0,0,0,0,0,0,0,0,0,0,...,1,0,4,0,0,0,0,2,0,3


## Mapping/ labeling the indicators

In [ ]:
# Load the mapping dictionary
mapping_dict_path = './data/Mapping Dictionary for Thesis_2021.py'
mapping_dict = {}
exec(open(mapping_dict_path).read(), mapping_dict)

# Map the categorical variables
for var in list_vars:
    if var in mapping_dict:
        df_21[var] = df_21[var].map(mapping_dict[var])

# Display the first few rows of the dataframe to verify the mapping
df_21.head()


,W_REGN,W_PROV,SEQUENCE_NO,FSIZE,REG_SAL,SEASON_SAL,WAGES,NETSHARE,CASH_ABROAD,CASH_DOMESTIC,...,LC36_AVAIL,LC37_WILLING,LC38_PREVJOB,LC39_YEAR,LC39_MONTH,LC41_POCC,LC42INDUSTRY,LC43_QKB,NEWEMPSTAT,PWGTPRV
0,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,NaN,1,185.0937
1,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,NaN,1,167.8637
2,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,,,2,,,,...,NaN,3,200.4619
3,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,NaN,1,192.4503
4,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,,...,NaN,,208.6799


## Domestic Helper Indicator Construction

This section creates a household-level indicator for domestic helpers:

**Steps:**
- **Dummy Creation:**  Use `pd.get_dummies()` on `LC03_REL` to generate a binary `LC03_REL_Domestic Helper` column.
- **Aggregation:**  Sum `LC03_REL_Domestic Helper` by `SEQUENCE_NO` to count helpers per household (`domestic_helper`).
- **Integration:**  Merge `domestic_helper` into `df_21` for household analysis.

**Outputs:**  
- `df_domestic_helper`: Households with helpers.  
- `domestic_helper`: Indicator in `df_21`.

In [107]:
create_dummies = ['LC03_REL']

# Create dummy variables for the specified columns
df_21_dummies = pd.get_dummies(df_21, columns=create_dummies, drop_first=True)

# List all the names of the newly created columns
new_columns = [col for col in df_21_dummies.columns if col not in df_21.columns]

df_21_dummies['domestic_helper'] = df_21_dummies.groupby('SEQUENCE_NO')['LC03_REL_Domestic Helper'].transform('sum')
df_21_dummies2 = df_21_dummies[['SEQUENCE_NO', 'domestic_helper']].drop_duplicates()
df_21 = df_21.merge(df_21_dummies2, on='SEQUENCE_NO', how='left')

# Display the first few rows of the updated dataframe
df_21


,W_REGN,W_PROV,SEQUENCE_NO,FSIZE,REG_SAL,SEASON_SAL,WAGES,NETSHARE,CASH_ABROAD,CASH_DOMESTIC,...,LC37_WILLING,LC38_PREVJOB,LC39_YEAR,LC39_MONTH,LC41_POCC,LC42INDUSTRY,LC43_QKB,NEWEMPSTAT,PWGTPRV,domestic_helper
0,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,...,NaN,1,185.0937,0
1,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,...,NaN,1,167.8637,0
2,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,,2,,,,...,NaN,3,200.4619,0
3,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,...,NaN,1,192.4503,0
4,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,,,,,,...,NaN,,208.6799,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90203,13,NCR-4th Dist.,130660,4.5,194394,15600,209994,0,0,12000,...,,,,,,...,NaN,,194.3881,0
90204,13,NCR-4th Dist.,130660,4.5,194394,15600,209994,0,0,12000,...,,,,,,...,NaN,,181.7025,0
90205,13,NCR-4th Dist.,130660,4.5,194394,15600,209994,0,0,12000,...,,,,,,...,NaN,,194.3881,0
90206,13,NCR-4th Dist.,130661,2.0,0,0,0,0,90000,134000,...,,2,,,,...,NaN,3,149.2373,0


## Household Head Sex and Marital Status Extraction

This step creates household-level indicators for the sex (`hh_sex`) and marital status (`hh_ms`) of the household head:

**Steps:**
 - **Select Columns:**  Extract `LC04_SEX`, `LC06_MSTAT`, `LC03_REL`, and `SEQUENCE_NO` from `df_21`.
 -  **Filter Heads:**  Keep only rows where `LC03_REL` is `'Head'`.
-  **Rename and Drop:**  Rename `LC04_SEX` to `hh_sex`, `LC06_MSTAT` to `hh_ms`, and drop `LC03_REL`.
-  **Merge:**   Merge these indicators back into `df_21` by `SEQUENCE_NO`.

**Result:**  
`df_21` now includes `hh_sex` and `hh_ms` for each household.

In [108]:
df_21_hh = df_21[['LC04_SEX',
    'LC06_MSTAT', 'LC03_REL', 'SEQUENCE_NO']]
df_21_hh = df_21_hh[df_21_hh['LC03_REL'] == 'Head']
df_21_hh.rename(columns={'LC04_SEX': 'hh_sex', 'LC06_MSTAT': 'hh_ms'}, inplace=True)
df_21_hh.drop(columns=['LC03_REL'], inplace=True)
df_21_hh

,hh_sex,hh_ms,SEQUENCE_NO
0,Male,Married,108973
7,Female,Married,108974
8,Male,Married,108975
12,Female,Married,108976
17,Male,Married,108977
...,...,...,...
90186,Male,Married,130657
90193,Male,Married,130658
90198,Female,Married,130659
90200,Female,Married,130660


In [109]:
# Merge df_21 with df_21_hh on 'SEQUENCE_NO'
df_21 = pd.merge(df_21, df_21_hh, on='SEQUENCE_NO', how='left')

# Display the first few rows of the updated dataframe to verify the changes
df_21.head()

,W_REGN,W_PROV,SEQUENCE_NO,FSIZE,REG_SAL,SEASON_SAL,WAGES,NETSHARE,CASH_ABROAD,CASH_DOMESTIC,...,LC39_YEAR,LC39_MONTH,LC41_POCC,LC42INDUSTRY,LC43_QKB,NEWEMPSTAT,PWGTPRV,domestic_helper,hh_sex,hh_ms
0,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,,,,...,NaN,1,185.0937,0,Male,Married
1,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,,,,...,NaN,1,167.8637,0,Male,Married
2,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,,,,...,NaN,3,200.4619,0,Male,Married
3,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,,,,...,NaN,1,192.4503,0,Male,Married
4,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,,,,...,NaN,,208.6799,0,Male,Married


## Refining the Unified DataFrame

This step prepares the final household-level dataset by:

- **Removing Individual Columns:**  
    Drops all columns starting with `LC`.

- **Excluding Unneeded Variables:**  
    Removes `NEWEMPSTAT` and `PWGTPRV`.

- **Adding Household Indicators:**  
    Merges aggregated indicators from `merged_hh_data` by `SEQUENCE_NO`.

- **Deduplicating:**  
    Removes duplicate rows for unique households.

**Result:**  
A clean DataFrame (`df_21`) with only household-level variables and indicators.

In [110]:
df_21 = df_21.loc[:, ~df_21.columns.str.startswith('LC')]
df_21 = df_21.drop(columns=['NEWEMPSTAT', 'PWGTPRV', 'C16_2DGT', 'C14_2DGT' ])

df_21 = df_21.merge(merged_hh_data, on='SEQUENCE_NO', how='left')



df_21 = df_21.drop_duplicates()

In [111]:
df_21['SEQUENCE_NO'].value_counts()

SEQUENCE_NO
108973    1
123428    1
123436    1
123435    1
123434    1
         ..
116201    1
116200    1
116199    1
116198    1
130661    1
Name: count, Length: 21684, dtype: int64

## Community Indicators: Average Barangay Metrics by Province

This section integrates community-level indicators into the household dataset by province. The workflow includes:

- **Loading the Community Indicators Dataset:**  
    Import barangay-level metrics from an external CSV file.

- **Filtering for Target Provinces:**  
    Select only the 17 Metro Manila provinces using their codes:
    - 801: Caloocan City
    - 802: City of Las Piñas
    - 803: City of Makati
    - 804: City of Malabon
    - 805: City of Mandaluyong
    - 806: City of Manila
    - 807: City of Marikina
    - 808: City of Muntinlupa
    - 809: City of Navotas
    - 810: City of Parañaque
    - 811: Pasay City
    - 812: City of Pasig
    - 813: Quezon City
    - 814: City of San Juan
    - 815: City of Taguig
    - 816: City of Valenzuela
    - 817: Pateros

- **Merging with Household Data:**  
    Attach average barangay-level indicators to each household based on province code.

**Purpose:**  
Enhance household-level analysis by incorporating contextual community metrics, enabling richer socioeconomic insights at the province level.

In [112]:
# --- mapping ----------------------------------------------------
name_to_3dig = {
    "Caloocan City": 801,
    "City of Las Piñas": 802,
    "City of Makati": 803,
    "City of Malabon": 804,
    "'City of Mandaluyong'": 805,
    "City of Manila": 806,
    "City of Marikina": 807,
    "City of Muntinlupa": 808,
    "City of Navotas": 809,
    "City of Parañaque": 810,
    "Pasay City": 811,
    "City of Pasig": 812,
    "Quezon City": 813,
    "City of San Juan": 814,
    "Taguig City": 815,
    "City of Valenzuela": 816,
    "Pateros": 817,
}

# --- example: rename column ------------------------------------
# df is your household / province dataframe
# assume the column that currently holds the names is called 'prov_name'
df_21["RPROV_new"] = (
    df_21["RPROV"]         
      .map(name_to_3dig)      
)

df_21['RPROV'] = df_21["RPROV_new"]
df_21['RPROV'].dtypes

dtype('float64')

In [ ]:
file_path = './data/output/pmt_comm_indicators_by_province.csv'

# Load the CSV file into a pandas DataFrame
df_comm_indicators = pd.read_csv(file_path)

df_21 = df_21.merge(df_comm_indicators, left_on='RPROV', right_on='PRV', how='left')
df_21 = df_21.dropna(subset=['RPROV'])

df_21


,W_REGN,W_PROV,SEQUENCE_NO,FSIZE,REG_SAL,SEASON_SAL,WAGES,NETSHARE,CASH_ABROAD,CASH_DOMESTIC,...,health,hospital,plaza,port,library,waterworks_system,cell_signal,landline,post_office,fire_station
0,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,0.168,0.028,0.079,0.013,0.010,0.999,1.0,1.0,0.025,0.058
1,13,Manila,108974,1.0,0,0,0,0,30000,3500,...,0.168,0.028,0.079,0.013,0.010,0.999,1.0,1.0,0.025,0.058
2,13,Manila,108975,4.0,0,0,0,0,0,42000,...,0.168,0.028,0.079,0.013,0.010,0.999,1.0,1.0,0.025,0.058
3,13,Manila,108976,5.0,259908,77328,337236,0,0,11500,...,0.168,0.028,0.079,0.013,0.010,0.999,1.0,1.0,0.025,0.058
4,13,Manila,108977,5.0,0,398656,398656,0,0,16800,...,0.168,0.028,0.079,0.013,0.010,0.999,1.0,1.0,0.025,0.058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21679,13,NCR-4th Dist.,130657,7.0,462544,0,462544,0,0,4000,...,1.000,0.214,0.643,0.000,0.536,1.000,1.0,1.0,0.143,0.607
21680,13,NCR-4th Dist.,130658,5.0,59800,5000,64800,0,0,9000,...,1.000,0.214,0.643,0.000,0.536,1.000,1.0,1.0,0.143,0.607
21681,13,NCR-4th Dist.,130659,1.5,192000,50000,242000,0,90000,10000,...,1.000,0.214,0.643,0.000,0.536,1.000,1.0,1.0,0.143,0.607
21682,13,NCR-4th Dist.,130660,4.5,194394,15600,209994,0,0,12000,...,1.000,0.214,0.643,0.000,0.536,1.000,1.0,1.0,0.143,0.607


## Categorical Variable Formatting

Key indicators are converted to categorical types for efficient analysis and modeling:

- Housing tenure (`H150104_TENURE_STA`)
- Roof material (`H150102_ROOF`)
- Wall material (`H150103_WALLS`)
- Building type (`H150101_BLDG_TYPE`)
- Water supply (`H150111_WATER_SUPPLY`)
- Toilet type (`H150109_TOILET`)
- Electricity access (`H150110_ELECTRICITY`)
- Province code (`RPROV`)
- Household head sex (`hh_sex`)
- Household head marital status (`hh_ms`)


In [114]:
# List of indicators to reformat as categories
indicators_to_category = [
    'H150104_TENURE_STA',
    'H150102_ROOF',
    'H150103_WALLS',
    'H150101_BLDG_TYPE',
    'H150111_WATER_SUPPLY',
    'H150109_TOILET',
    'H150110_ELECTRICITY',
    'RPROV',
    'hh_sex',
    'hh_ms'
]

# Convert the specified columns to category type
df_21[indicators_to_category] = df_21[indicators_to_category].astype('category')

## Saving the Cleaned DataFrame

The cleaned DataFrame `df_21` is saved as `df_21_ols.csv` in the `output` folder for further analysis.


In [115]:
df_21.to_csv(os.path.join(output_folder, 'df_21_ols.csv'), index=False)
df_21

,W_REGN,W_PROV,SEQUENCE_NO,FSIZE,REG_SAL,SEASON_SAL,WAGES,NETSHARE,CASH_ABROAD,CASH_DOMESTIC,...,health,hospital,plaza,port,library,waterworks_system,cell_signal,landline,post_office,fire_station
0,13,Manila,108973,6.5,337000,0,337000,0,0,2000,...,0.168,0.028,0.079,0.013,0.010,0.999,1.0,1.0,0.025,0.058
1,13,Manila,108974,1.0,0,0,0,0,30000,3500,...,0.168,0.028,0.079,0.013,0.010,0.999,1.0,1.0,0.025,0.058
2,13,Manila,108975,4.0,0,0,0,0,0,42000,...,0.168,0.028,0.079,0.013,0.010,0.999,1.0,1.0,0.025,0.058
3,13,Manila,108976,5.0,259908,77328,337236,0,0,11500,...,0.168,0.028,0.079,0.013,0.010,0.999,1.0,1.0,0.025,0.058
4,13,Manila,108977,5.0,0,398656,398656,0,0,16800,...,0.168,0.028,0.079,0.013,0.010,0.999,1.0,1.0,0.025,0.058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21679,13,NCR-4th Dist.,130657,7.0,462544,0,462544,0,0,4000,...,1.000,0.214,0.643,0.000,0.536,1.000,1.0,1.0,0.143,0.607
21680,13,NCR-4th Dist.,130658,5.0,59800,5000,64800,0,0,9000,...,1.000,0.214,0.643,0.000,0.536,1.000,1.0,1.0,0.143,0.607
21681,13,NCR-4th Dist.,130659,1.5,192000,50000,242000,0,90000,10000,...,1.000,0.214,0.643,0.000,0.536,1.000,1.0,1.0,0.143,0.607
21682,13,NCR-4th Dist.,130660,4.5,194394,15600,209994,0,0,12000,...,1.000,0.214,0.643,0.000,0.536,1.000,1.0,1.0,0.143,0.607
